In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())


In [ ]:
!nvidia-smi

In [25]:
%cd fast_align
%mkdir build
%cd build
%cmake ..
%make


[Errno 2] No such file or directory: 'fast_align'
/workspace/fast_align/build
/workspace/fast_align/build/build


UsageError: Line magic function `%cmake` not found.


In [25]:
import pandas as pd


data = pd.read_csv('tokenized_data.csv')

# Prepare data for FastAlign
with open('fast_align/sample_data/tokenized_parallel_corpus.txt', 'w', encoding='utf-8') as f:
    for index, row in data.iterrows():
        english_tokens = ' '.join(eval(row['English_Tokens']))  # Ensure tokens are properly extracted
        bangla_tokens = ' '.join(eval(row['Bangla_Tokens']))
        f.write(f"{english_tokens} ||| {bangla_tokens}\n")


In [28]:
!chmod +x /workspace/fast_align/build/fast_align

In [29]:
!ls

Llama_3_1_8b_+_Unsloth_2x_faster_finetuning.ipynb  fast_align
axolotl						   mistral_finetune_7b.ipynb
axolotl-update					   mistral_finetune_api.ipynb
ben-ben2017.txt					   test.ipynb
data						   tokenized_data.csv
eng-enggnv.txt					   transtokenizers.py


In [30]:
%cd /workspace/fast_align/build


/workspace/fast_align/build


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [37]:
%cd ..

/workspace


In [39]:
%cd workspace/fast_align/build && ./fast_align -i workspace/data/tokenized_parallel_corpus.txt -d -o -v > workspace/data/alignments.txt

[Errno 2] No such file or directory: 'workspace/fast_align/build && ./fast_align -i workspace/data/tokenized_parallel_corpus.txt -d -o -v > workspace/data/alignments.txt'
/workspace


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [40]:
!ls /workspace/fast_align/build
!ls /workspace/data


CMakeCache.txt	Makefile  cmake_install.cmake  force_align.py
CMakeFiles	atools	  fast_align
tokenized_parallel_corpus.txt


In [41]:
!chmod +x /workspace/fast_align/build/fast_align

In [42]:
!cd /workspace/fast_align/build && ./fast_align -i /workspace/data/tokenized_parallel_corpus.txt -d -o -v > /workspace/data/alignments.txt

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 
expected target length = source length * 1.00528
ITERATION 1
  log_e likelihood: -5263.71
  log_2 likelihood: -7593.93
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.165861
       size counts: 9
ITERATION 2
  log_e likelihood: -1911.06
  log_2 likelihood: -2757.07
     cross entropy: 10.8546
        perplexity: 1851.69
      posterior p0: 0.0776098
 posterior al-feat: -0.123301
       size counts: 9
  1  model al-feat: -0.176679 (tension=4)
  2  model al-feat: -0.152061 (tension=5.06757)
  3  model al-feat: -0.141014 (tension=5.64277)
  4  model al-feat: -0.134871 (tension=5.99704)
  5  model al-feat: -0.131105 (tension=6.22844)
  6  model al-feat: -0.128668 (tension=6.38453)
  7  model al-feat: -0.127038 (tension=6.49187)
  8  model al-feat: -0.125925 (tension=6.56663)
     final tension: 6.61911
ITERATION 3
  log_e likelihood: -1197.63
  log_2 likelihood: -1727.82
     cross entropy: 6.80243


In [56]:
## Starting creation of parallel corpus 

In [57]:
# Read the downloaded files
with open("ben-ben2017.txt", "r", encoding="utf-8") as f:
    bangla_sentences = f.readlines()

with open("eng-enggnv.txt", "r", encoding="utf-8") as f:
    english_sentences = f.readlines()

# Ensure both files have the same number of lines for a 1-to-1 alignment
assert len(bangla_sentences) == len(english_sentences), "Mismatch in sentence count."

# Combine into a parallel corpus format
with open("parallel_corpus.txt", "w", encoding="utf-8") as f:
    for en, bn in zip(english_sentences, bangla_sentences):
        f.write(f"{en.strip()} ||| {bn.strip()}\n")


In [59]:
#Tokenization

In [58]:
from transformers import AutoTokenizer

# Load tokenizers
english_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
bangla_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

# Tokenize the corpus
with open("tokenized_parallel_corpus.txt", "w", encoding="utf-8") as out_file:
    with open("parallel_corpus.txt", "r", encoding="utf-8") as in_file:
        for line in in_file:
            en, bn = line.split(" ||| ")
            en_tokens = " ".join(english_tokenizer.tokenize(en.strip()))
            bn_tokens = " ".join(bangla_tokenizer.tokenize(bn.strip()))
            out_file.write(f"{en_tokens} ||| {bn_tokens}\n")


In [60]:
#Token Alignment with FastAlign

In [65]:
!chmod +x fast_align/build/fast_align

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [67]:
! fast_align/build/fast_align -i tokenized_parallel_corpus.txt -d -o -v > alignments.txt

ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...Error in line 3167
 ||| শব ##িত ##র ##রে ##াগের , পে ##াশা ##কে অব ##স ##থি ##ত কষ ##ঠে ##র ও গহ ##ের
.Error in line 4491
 ||| 
.Error in line 5952
 ||| তখন যি ##হে ##াশ ##য সেই রকম করলেন । সেই দিনের ইসরা ##যে ##ল - সন ##তান ##দের জন ##য যি ##রী ##হে ##া নগর সম ##পর ##ণ ##র ##পে বনধ ছিল , কেউ ভিতরে আসত না , কেউ বাইরে যেত না ।
..Error in line 7774
▁Therefore ▁I on athan ▁said ▁to ▁D au id , ▁Go e ▁in ▁peace : ▁that ▁which ▁we ▁ha ue ▁sw orne ▁both ▁of ▁vs ▁in ▁the ▁Name ▁of ▁the ▁Lord , ▁saying , ▁The ▁Lord ▁be ▁between e ▁me ▁and ▁the e , ▁and ▁between e ▁my ▁se ede ▁and ▁between e ▁thy ▁se ede , ▁let ▁it ▁stand ▁for ▁e uer . ||| 
.Error in line 8052
 ||| হায ! ( শক ##তিশা ##লী য ##দ ##ধ ) বীরের ##া পরাজিত হল , য ##দ ##ধের সমস ##ত অস ##তর ##গলি ধবংস হল ! ”
.Error in line 9526
 ||| 
.Error in line 10728
 ||| 
..Error in line 12751
 ||| তাতে খে ##াজ খবর নি ##যে সেই কথা পরম ##াণ হলে ঐ দ ##জনকে গাছে ফাসি দেও ##যা হল এবং সেই কথা রাজার সামনে ইতিহাস বইতে লেখা হল ।
.Error in line 13012
 |||

In [68]:
## to many errors trying to clean the data 

In [69]:
# Function to clean and prepare the parallel corpus
def clean_parallel_corpus(english_path, bangla_path, output_path):
    with open(english_path, "r", encoding="utf-8") as en_file, open(bangla_path, "r", encoding="utf-8") as bn_file, open(output_path, "w", encoding="utf-8") as out_file:
        for en_line, bn_line in zip(en_file, bn_file):
            en_line = en_line.strip()
            bn_line = bn_line.strip()
            if en_line and bn_line:  # Ensure both lines are not empty
                out_file.write(f"{en_line} ||| {bn_line}\n")


clean_parallel_corpus("eng-enggnv.txt", "ben-ben2017.txt", "cleaned_parallel_corpus.txt")


In [70]:
#tokenizing clean parallel corpus 

from transformers import AutoTokenizer

# Load the English and Bangla tokenizers
english_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")
bangla_tokenizer = AutoTokenizer.from_pretrained("sagorsarker/bangla-bert-base")

# Function to tokenize the parallel corpus
def tokenize_parallel_corpus(input_path, output_path):
    with open(input_path, "r", encoding="utf-8") as in_file, open(output_path, "w", encoding="utf-8") as out_file:
        for line in in_file:
            en_sentence, bn_sentence = line.split(" ||| ")
            en_tokens = " ".join(english_tokenizer.tokenize(en_sentence.strip()))
            bn_tokens = " ".join(bangla_tokenizer.tokenize(bn_sentence.strip()))
            out_file.write(f"{en_tokens} ||| {bn_tokens}\n")

# Tokenize the cleaned parallel corpus
tokenize_parallel_corpus("cleaned_parallel_corpus.txt", "tokenized_parallel_corpus.txt")


In [71]:
#running fast align 

In [72]:
# Assuming FastAlign binary is executable and in the correct path
!fast_align/build/fast_align -i tokenized_parallel_corpus.txt -d -o -v > alignments.txt


ARG=i
ARG=d
ARG=o
ARG=v
INITIAL PASS 


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


..............................
expected target length = source length * 0.976589
ITERATION 1
..............................
  log_e likelihood: -2.37491e+07
  log_2 likelihood: -3.42628e+07
     cross entropy: 29.8974
        perplexity: 1e+09
      posterior p0: 0.08
 posterior al-feat: -0.16413
       size counts: 3484
ITERATION 2
..............................
  log_e likelihood: -7.21468e+06
  log_2 likelihood: -1.04086e+07
     cross entropy: 9.08242
        perplexity: 542.104
      posterior p0: 0.0827937
 posterior al-feat: -0.148929
       size counts: 3484
  1  model al-feat: -0.180298 (tension=4)
  2  model al-feat: -0.164924 (tension=4.62739)
  3  model al-feat: -0.157869 (tension=4.94729)
  4  model al-feat: -0.154138 (tension=5.1261)
  5  model al-feat: -0.152032 (tension=5.23029)
  6  model al-feat: -0.1508 (tension=5.29235)
  7  model al-feat: -0.150065 (tension=5.32976)
  8  model al-feat: -0.149622 (tension=5.35249)
     final tension: 5.36635
ITERATION 3
............

In [73]:
#token mapping 

In [ ]:
# Example function to create token mapping

from transformers import AutoModel, AutoTokenizer

# Load the pre-trained model and tokenizer for English
english_model_name = "mistralai/Mistral-7B-v0.1"
english_model = AutoModel.from_pretrained(english_model_name)
english_tokenizer = AutoTokenizer.from_pretrained(english_model_name)

# Load the pre-trained model and tokenizer for Bangla
bangla_model_name = "sagorsarker/bangla-bert-base"
bangla_model = AutoModel.from_pretrained(bangla_model_name)
bangla_tokenizer = AutoTokenizer.from_pretrained(bangla_model_name)

def create_token_mapping(alignments_file):
    token_mapping = {}
    with open(alignments_file, "r") as file:
        for line in file:
            alignment_pairs = line.strip().split()
            for pair in alignment_pairs:
                en_index, bn_index = map(int, pair.split('-'))
                if en_index not in token_mapping:
                    token_mapping[en_index] = []
                token_mapping[en_index].append(bn_index)
    return token_mapping

# Example function to adapt embeddings
def adapt_embeddings(english_embedding_matrix, bangla_embedding_matrix, token_mapping):
    for en_token, bn_tokens in token_mapping.items():
        if en_token < len(english_embedding_matrix):
            en_embedding = english_embedding_matrix[en_token]
            if bn_tokens:
                avg_embedding = np.mean([bangla_embedding_matrix[bn_token] for bn_token in bn_tokens if bn_token < len(bangla_embedding_matrix)], axis=0)
                bangla_embedding_matrix[bn_tokens[0]] = avg_embedding  # Update the first Bangla token's embedding
    return bangla_embedding_matrix

# Sample usage
english_embedding_matrix = english_model.get_input_embeddings().weight.detach().numpy()
bangla_embedding_matrix = bangla_model.get_input_embeddings().weight.detach().numpy()
token_mapping = create_token_mapping("alignments.txt")
adapted_embeddings = adapt_embeddings(english_embedding_matrix, bangla_embedding_matrix, token_mapping)


In [74]:
#another code - 

In [ ]:
from transformers import AutoModel, AutoTokenizer
import numpy as np

# Load the pre-trained model and tokenizer for English
english_model_name = "mistralai/Mistral-7B-v0.1"
english_model = AutoModel.from_pretrained(english_model_name)
english_tokenizer = AutoTokenizer.from_pretrained(english_model_name)

# Load the pre-trained model and tokenizer for Bangla
bangla_model_name = "sagorsarker/bangla-bert-base"
bangla_model = AutoModel.from_pretrained(bangla_model_name)
bangla_tokenizer = AutoTokenizer.from_pretrained(bangla_model_name)

def create_token_mapping(alignments_file):
    token_mapping = {}
    with open(alignments_file, "r") as file:
        for line in file:
            alignment_pairs = line.strip().split()
            for pair in alignment_pairs:
                en_index, bn_index = map(int, pair.split('-'))
                if en_index not in token_mapping:
                    token_mapping[en_index] = []
                token_mapping[en_index].append(bn_index)
    return token_mapping

# Example function to adapt embeddings
def adapt_embeddings(english_embedding_matrix, bangla_embedding_matrix, token_mapping):
    for en_token, bn_tokens in token_mapping.items():
        if en_token < len(english_embedding_matrix):
            en_embedding = english_embedding_matrix[en_token]
            if bn_tokens:
                # Use the average embedding for Bangla tokens
                avg_embedding = np.mean(
                    [bangla_embedding_matrix[bn_token] for bn_token in bn_tokens if bn_token < len(bangla_embedding_matrix)], 
                    axis=0
                )
                # Assign the average embedding to each corresponding Bangla token
                for bn_token in bn_tokens:
                    if bn_token < len(bangla_embedding_matrix):
                        bangla_embedding_matrix[bn_token] = avg_embedding
    return bangla_embedding_matrix

# Sample usage
english_embedding_matrix = english_model.get_input_embeddings().weight.detach().numpy()
bangla_embedding_matrix = bangla_model.get_input_embeddings().weight.detach().numpy()
token_mapping = create_token_mapping("alignments.txt")
adapted_embeddings = adapt_embeddings(english_embedding_matrix, bangla_embedding_matrix, token_mapping)


In [ ]:
!pip install torch==2.2.1
!pip install transformers==4.38.1

In [4]:
import torch
print(torch.__version__)


2.2.1+cu121


In [5]:
import transformers
print(transformers.__version__)

4.38.1
